# Numerical SOS decomposition

## Preliminaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ncpol2sdpa as ncp
import json

# Define quantum operators
A_config = [2,2]
B_config = [2,2]

# Operators in problem
A = [Ai for Ai in ncp.generate_measurements(A_config, 'A')]
B = [Bj for Bj in ncp.generate_measurements(B_config, 'B')]

ops = ncp.flatten([A,B])        # Base monomials involved in problem
subs = {}

# 1. Unitarity: A^2 = 1
for op in ops:
    subs[op**2] = 1

# 2. Commutation: force a canonical ordering B*A → A*B
for Ai in ncp.flatten(A):
    for Bj in ncp.flatten(B):
        subs[Bj * Ai] = Ai * Bj

## Test: the CHSH expression

In [2]:
moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = []                       # extra monomials

obj = (A[0][0]*B[0][0] + A[1][0]*B[0][0] + A[0][0]*B[1][0] - A[1][0]*B[1][0])/(2*np.sqrt(2))

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=5,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)

1.0000000000810259 0.9999999999441891 optimal


## Testing Function

In [3]:
def test_point_quantum_bound(theta, r0, r1, level=0, verbose=False):
    """
    Tests whether a point (parametrized by theta, r0 = p1, r1 = 2*p4) is quantum-realizable
    by solving the associated SDP relaxation at level 0 with different sets of monomials.

    Parameters:
    -----------
    theta : float
        The measurement angle θ in radians.
    r0 : float
        Parameter such that p1 = r0.
    r1 : float
        Parameter such that p4 = 0.5 * r1.
    level : str or int
        Level of the relaxation: 0, 'AB', 'ABB', or 'AAB'.
    solver : str
        SDP solver to use (e.g., "mosek", "cvxopt", "sdpa").
    verbose : bool
        Whether to print verbose SDP solver output.

    Returns:
    --------
    primal : float
        The negative primal bound.
    dual : float
        The negative dual bound.
    status : str
        The solver status string.
    """

    s = np.sqrt(2)
    sin_2T = np.sin(2 * theta)
    sin_4T = np.sin(4 * theta)
    cos_2T = np.cos(2 * theta)
    cos_4T = np.cos(4 * theta)
    cos_6T = np.cos(6 * theta)

    # Define objective expression
    obj = (-1/(2*(3 - cos_4T))*0.5*r1*(-4*B[1][0]*(-3 + cos_4T) + s*(-5*cos_2T + cos_6T)*A[0][0]*B[1][0] 
        + s*(-5*cos_2T + cos_6T)*A[1][0]*B[1][0] + 2*s*A[0][0]*(-3 + cos_4T)*sin_2T - 2*s*A[1][0]*(-3 + cos_4T)*sin_2T) 
        - 1/(2*(3 - cos_4T))*(-4*B[0][0]*cos_2T - s*A[0][0]*B[0][0] + s*cos_4T*A[0][0]*B[0][0] - s*A[1][0]*B[0][0] 
        + s*cos_4T*A[1][0]*B[0][0] - 2*s*A[0][0]*B[1][0]*sin_2T + 2*s*A[1][0]*B[1][0]*sin_2T) 
        - 1/(2*(3 - cos_4T))*r0*(s*A[0][0]*(-3 + cos_4T) + s*A[1][0]*(-3 + cos_4T) - 4*B[0][0]*(-1 + cos_4T) 
        + 2*s*cos_2T*A[0][0]*B[0][0] + 2*s*cos_2T*A[1][0]*B[0][0] - 2*s*cos_2T*A[0][0]*B[1][0]*sin_2T + s*A[1][0]*B[1][0]*sin_4T))

    def get_extra_monomials(level):
        """
        Generate extra monomials for a given level string such as:
        'AA', 'BB', 'AB', 'AAB', 'ABB', 'AA+BB+AB+AAB+ABB', etc.
        """
        if level in [0, '0', None]:
            return []

        level_parts = level.split('+') if isinstance(level, str) else []
        monos = []
        Aflat = ncp.flatten(A)
        Bflat = ncp.flatten(B)

        if 'AA' in level_parts:
            for a1 in Aflat:
                for a2 in Aflat:
                    monos.append(a1 * a2)

        if 'BB' in level_parts:
            for b1 in Bflat:
                for b2 in Bflat:
                    monos.append(b1 * b2)

        if 'AB' in level_parts:
            for a in Aflat:
                for b in Bflat:
                    monos.append(a * b)

        if 'AAB' in level_parts:
            for a1 in Aflat:
                for a2 in Aflat:
                    for b in Bflat:
                        monos.append(a1 * a2 * b)

        if 'ABB' in level_parts:
            for a in Aflat:
                for b1 in Bflat:
                    for b2 in Bflat:
                        monos.append(a * b1 * b2)

        return monos[:]

    extra_monos = get_extra_monomials(level)

    # Create and solve SDP relaxation at fixed level=0
    sdp = ncp.SdpRelaxation(ops, verbose=int(verbose), normalized=True, parallel=0)
    sdp.get_relaxation(
        level=0,
        equalities=op_eqs[:],
        inequalities=op_ineqs[:],
        momentequalities=moment_eqs[:],
        momentinequalities=moment_ineqs[:],
        objective=-obj,
        substitutions=subs,
        extramonomials=extra_monos
    )
    sdp.solve(solver="mosek")

    return -sdp.primal, -sdp.dual, sdp.status

## Testing Vertices

In [4]:
# Load the JSON file
with open("vertexCoords.json", "r") as f:
    data = json.load(f)

In [6]:
theta = np.pi / 4
tolerance = 1e-6
levels = [0, 'AA', 'BB', 'AB', 'BB+AB', 'ABB', 'AAB', 'AB+ABB', 'AA+AB+ABB', 'BB+AB+ABB', 'AA+BB+AB+ABB', 'AB+AAB+ABB']

with open("vertexCoords.json", "r") as f:
    local_vertices_data = json.load(f)

local_verts = None
for theta_value, verts in local_vertices_data:
    if abs(theta_value - theta) < tolerance:
        local_verts = np.array(verts)
        break

if local_verts is None:
    raise ValueError(f"No local polytope found for theta = {theta}")

print(f"Found {len(local_verts)} local vertices for θ = {theta:.6f}")

for i, (r0, r1) in enumerate(local_verts):
    print(f"\n--- Vertex #{i+1}: (r0 = {r0:.6f}, r1 = {r1:.6f}) ---")
    for lvl in levels:
        p, d, s = test_point_quantum_bound(theta, r0, r1, level=lvl)
        print(f"  Level {lvl:<12}: primal = {p:.10f}, dual = {d:.10f}, status = {s}")

Found 8 local vertices for θ = 0.785398

--- Vertex #1: (r0 = 0.000000, r1 = 0.292893) ---
  Level 0           : primal = 1.0857864377, dual = 1.0857864365, status = optimal
  Level AA          : primal = 1.0857864379, dual = 1.0857864367, status = optimal
  Level BB          : primal = 1.0857864383, dual = 1.0857864346, status = optimal
  Level AB          : primal = 1.0091153569, dual = 1.0091153431, status = optimal
  Level BB+AB       : primal = 1.0091153531, dual = 1.0091153512, status = optimal
  Level ABB         : primal = 1.0857864420, dual = 1.0857864355, status = optimal
  Level AAB         : primal = 1.0828606998, dual = 1.0828606844, status = optimal
  Level AB+ABB      : primal = 1.0000000084, dual = 0.9999999940, status = optimal
  Level AA+AB+ABB   : primal = 1.0000000012, dual = 0.9999999992, status = optimal
  Level BB+AB+ABB   : primal = 1.0000000014, dual = 0.9999999990, status = optimal
  Level AA+BB+AB+ABB: primal = 1.0000000006, dual = 0.9999999993, status = opti

In [7]:
theta = np.pi / 8 + 7*np.pi / (64)
tolerance = 1e-6
levels = [0, 'AA', 'BB', 'AB', 'BB+AB', 'ABB', 'AAB', 'AB+ABB', 'AA+AB+ABB', 'BB+AB+ABB', 'AA+BB+AB+ABB', 'AB+AAB+ABB']


with open("vertexCoords.json", "r") as f:
    local_vertices_data = json.load(f)

local_verts = None
for theta_value, verts in local_vertices_data:
    if abs(theta_value - theta) < tolerance:
        local_verts = np.array(verts)
        break

if local_verts is None:
    raise ValueError(f"No local polytope found for theta = {theta}")

print(f"Found {len(local_verts)} local vertices for θ = {theta:.6f}")

for i, (r0, r1) in enumerate(local_verts):
    print(f"\n--- Vertex #{i+1}: (r0 = {r0:.6f}, r1 = {r1:.6f}) ---")
    for lvl in levels:
        p, d, s = test_point_quantum_bound(theta, r0, r1, level=lvl)
        print(f"  Level {lvl:<12}: primal = {p:.10f}, dual = {d:.10f}, status = {s}")

Found 8 local vertices for θ = 0.736311

--- Vertex #1: (r0 = -0.019962, r1 = 0.294916) ---
  Level 0           : primal = 1.1173009003, dual = 1.1173008978, status = optimal
  Level AA          : primal = 1.1100970986, dual = 1.1100970950, status = optimal
  Level BB          : primal = 1.1146679937, dual = 1.1146679865, status = optimal
  Level AB          : primal = 1.0097272452, dual = 1.0097272426, status = optimal
  Level BB+AB       : primal = 1.0097272464, dual = 1.0097272422, status = optimal
  Level ABB         : primal = 1.0957463807, dual = 1.0957463766, status = optimal
  Level AAB         : primal = 1.0893883795, dual = 1.0893883667, status = optimal
  Level AB+ABB      : primal = 1.0000606163, dual = 1.0000606142, status = optimal
  Level AA+AB+ABB   : primal = 1.0000000020, dual = 0.9999999986, status = optimal
  Level BB+AB+ABB   : primal = 1.0000000011, dual = 0.9999999993, status = optimal
  Level AA+BB+AB+ABB: primal = 1.0000000009, dual = 0.9999999990, status = opt

In [8]:
theta = np.pi / 8 + 6*np.pi / (64)
tolerance = 1e-6
levels = [0, 'AA', 'BB', 'AB', 'BB+AB', 'ABB', 'AAB', 'AB+ABB', 'AA+AB+ABB', 'BB+AB+ABB', 'AA+BB+AB+ABB', 'AB+AAB+ABB']

with open("vertexCoords.json", "r") as f:
    local_vertices_data = json.load(f)

local_verts = None
for theta_value, verts in local_vertices_data:
    if abs(theta_value - theta) < tolerance:
        local_verts = np.array(verts)
        break

if local_verts is None:
    raise ValueError(f"No local polytope found for theta = {theta}")

print(f"Found {len(local_verts)} local vertices for θ = {theta:.6f}")

for i, (r0, r1) in enumerate(local_verts):
    print(f"\n--- Vertex #{i+1}: (r0 = {r0:.6f}, r1 = {r1:.6f}) ---")
    for lvl in levels:
        p, d, s = test_point_quantum_bound(theta, r0, r1, level=lvl)
        print(f"  Level {lvl:<12}: primal = {p:.10f}, dual = {d:.10f}, status = {s}")

Found 8 local vertices for θ = 0.687223

--- Vertex #1: (r0 = -0.037597, r1 = 0.301323) ---
  Level 0           : primal = 1.1491171455, dual = 1.1491171406, status = optimal
  Level AA          : primal = 1.1365835595, dual = 1.1365835530, status = optimal
  Level BB          : primal = 1.1461564028, dual = 1.1461563978, status = optimal
  Level AB          : primal = 1.0117711974, dual = 1.0117711782, status = optimal
  Level BB+AB       : primal = 1.0117711873, dual = 1.0117711838, status = optimal
  Level ABB         : primal = 1.1068884791, dual = 1.1068884759, status = optimal
  Level AAB         : primal = 1.1068173119, dual = 1.1068173081, status = optimal
  Level AB+ABB      : primal = 1.0005096592, dual = 1.0005096565, status = optimal
  Level AA+AB+ABB   : primal = 1.0000000042, dual = 0.9999999972, status = optimal
  Level BB+AB+ABB   : primal = 1.0000000052, dual = 0.9999999960, status = optimal
  Level AA+BB+AB+ABB: primal = 1.0000000021, dual = 0.9999999978, status = opt

In [10]:
theta = np.pi / 8 + 5*np.pi / (64)
tolerance = 1e-6
levels = [0, 'AA', 'BB', 'AB', 'BB+AB', 'ABB', 'AAB', 'AB+ABB', 'AA+AB+ABB', 'BB+AB+ABB', 'AA+BB+AB+ABB', 'AB+AAB+ABB']

with open("vertexCoords.json", "r") as f:
    local_vertices_data = json.load(f)

local_verts = None
for theta_value, verts in local_vertices_data:
    if abs(theta_value - theta) < tolerance:
        local_verts = np.array(verts)
        break

if local_verts is None:
    raise ValueError(f"No local polytope found for theta = {theta}")

print(f"Found {len(local_verts)} local vertices for θ = {theta:.6f}")

for i, (r0, r1) in enumerate(local_verts):
    print(f"\n--- Vertex #{i+1}: (r0 = {r0:.6f}, r1 = {r1:.6f}) ---")
    for lvl in levels:
        p, d, s = test_point_quantum_bound(theta, r0, r1, level=lvl)
        print(f"  Level {lvl:<12}: primal = {p:.10f}, dual = {d:.10f}, status = {s}")

Found 6 local vertices for θ = 0.638136

--- Vertex #1: (r0 = 0.383354, r1 = 0.000000) ---
  Level 0           : primal = 1.1658943478, dual = 1.1658943272, status = optimal
  Level AA          : primal = 1.1480300095, dual = 1.1480300012, status = optimal
  Level BB          : primal = 1.1658943401, dual = 1.1658943297, status = optimal
  Level AB          : primal = 1.0122647413, dual = 1.0122647327, status = optimal
  Level BB+AB       : primal = 1.0122647422, dual = 1.0122647339, status = optimal
  Level ABB         : primal = 1.1280058457, dual = 1.1280058378, status = optimal
  Level AAB         : primal = 1.0763199724, dual = 1.0763199661, status = optimal
  Level AB+ABB      : primal = 1.0000000047, dual = 0.9999999946, status = optimal
  Level AA+AB+ABB   : primal = 1.0000000061, dual = 0.9999999929, status = optimal
  Level BB+AB+ABB   : primal = 1.0000000015, dual = 0.9999999989, status = optimal
  Level AA+BB+AB+ABB: primal = 1.0000000015, dual = 0.9999999989, status = opti

In [11]:
theta = np.pi / 8 + 4*np.pi / (64)
tolerance = 1e-6
levels = [0, 'AA', 'BB', 'AB', 'BB+AB', 'ABB', 'AAB', 'AB+ABB', 'AA+AB+ABB', 'BB+AB+ABB', 'AA+BB+AB+ABB', 'AB+AAB+ABB']

with open("vertexCoords.json", "r") as f:
    local_vertices_data = json.load(f)

local_verts = None
for theta_value, verts in local_vertices_data:
    if abs(theta_value - theta) < tolerance:
        local_verts = np.array(verts)
        break

if local_verts is None:
    raise ValueError(f"No local polytope found for theta = {theta}")

print(f"Found {len(local_verts)} local vertices for θ = {theta:.6f}")

for i, (r0, r1) in enumerate(local_verts):
    print(f"\n--- Vertex #{i+1}: (r0 = {r0:.6f}, r1 = {r1:.6f}) ---")
    for lvl in levels:
        p, d, s = test_point_quantum_bound(theta, r0, r1, level=lvl)
        print(f"  Level {lvl:<12}: primal = {p:.10f}, dual = {d:.10f}, status = {s}")

Found 6 local vertices for θ = 0.589049

--- Vertex #1: (r0 = 0.421218, r1 = 0.000000) ---
  Level 0           : primal = 1.2089273707, dual = 1.2089273633, status = optimal
  Level AA          : primal = 1.1720264585, dual = 1.1720264438, status = optimal
  Level BB          : primal = 1.2089273714, dual = 1.2089273658, status = optimal
  Level AB          : primal = 1.0126383879, dual = 1.0126383815, status = optimal
  Level BB+AB       : primal = 1.0126383922, dual = 1.0126383802, status = optimal
  Level ABB         : primal = 1.1488761855, dual = 1.1488761783, status = optimal
  Level AAB         : primal = 1.0809695991, dual = 1.0809695789, status = optimal
  Level AB+ABB      : primal = 1.0000000053, dual = 0.9999999939, status = optimal
  Level AA+AB+ABB   : primal = 1.0000000069, dual = 0.9999999957, status = optimal
  Level BB+AB+ABB   : primal = 1.0000000017, dual = 0.9999999987, status = optimal
  Level AA+BB+AB+ABB: primal = 1.0000000050, dual = 0.9999999960, status = opti

In [13]:
theta = np.pi / 8 + 3*np.pi / (64)
tolerance = 1e-6
levels = [0, 'AA', 'BB', 'AB', 'BB+AB', 'ABB', 'AAB', 'AB+ABB', 'AA+AB+ABB', 'BB+AB+ABB', 'AA+BB+AB+ABB', 'AB+AAB+ABB']

with open("vertexCoords.json", "r") as f:
    local_vertices_data = json.load(f)

local_verts = None
for theta_value, verts in local_vertices_data:
    if abs(theta_value - theta) < tolerance:
        local_verts = np.array(verts)
        break

if local_verts is None:
    raise ValueError(f"No local polytope found for theta = {theta}")

print(f"Found {len(local_verts)} local vertices for θ = {theta:.6f}")

for i, (r0, r1) in enumerate(local_verts):
    print(f"\n--- Vertex #{i+1}: (r0 = {r0:.6f}, r1 = {r1:.6f}) ---")
    for lvl in levels:
        p, d, s = test_point_quantum_bound(theta, r0, r1, level=lvl)
        print(f"  Level {lvl:<12}: primal = {p:.10f}, dual = {d:.10f}, status = {s}")

Found 6 local vertices for θ = 0.539961

--- Vertex #1: (r0 = 0.467438, r1 = 0.000000) ---
  Level 0           : primal = 1.2536100256, dual = 1.2536100109, status = optimal
  Level AA          : primal = 1.1944582202, dual = 1.1944582145, status = optimal
  Level BB          : primal = 1.2536100270, dual = 1.2536100097, status = optimal
  Level AB          : primal = 1.0122457871, dual = 1.0122457844, status = optimal
  Level BB+AB       : primal = 1.0122457927, dual = 1.0122457810, status = optimal
  Level ABB         : primal = 1.1776818754, dual = 1.1776818630, status = optimal
  Level AAB         : primal = 1.0945789478, dual = 1.0945789441, status = optimal
  Level AB+ABB      : primal = 1.0000000042, dual = 0.9999999949, status = optimal
  Level AA+AB+ABB   : primal = 1.0000000080, dual = 0.9999999934, status = optimal
  Level BB+AB+ABB   : primal = 1.0000000021, dual = 0.9999999984, status = optimal
  Level AA+BB+AB+ABB: primal = 1.0000000107, dual = 0.9999999913, status = opti

In [14]:
theta = np.pi / 8 + 2*np.pi / (64)
tolerance = 1e-6
levels = [0, 'AA', 'BB', 'AB', 'BB+AB', 'ABB', 'AAB', 'AB+ABB', 'AA+AB+ABB', 'BB+AB+ABB', 'AA+BB+AB+ABB', 'AB+AAB+ABB']

with open("vertexCoords.json", "r") as f:
    local_vertices_data = json.load(f)

local_verts = None
for theta_value, verts in local_vertices_data:
    if abs(theta_value - theta) < tolerance:
        local_verts = np.array(verts)
        break

if local_verts is None:
    raise ValueError(f"No local polytope found for theta = {theta}")

print(f"Found {len(local_verts)} local vertices for θ = {theta:.6f}")

for i, (r0, r1) in enumerate(local_verts):
    print(f"\n--- Vertex #{i+1}: (r0 = {r0:.6f}, r1 = {r1:.6f}) ---")
    for lvl in levels:
        p, d, s = test_point_quantum_bound(theta, r0, r1, level=lvl)
        print(f"  Level {lvl:<12}: primal = {p:.10f}, dual = {d:.10f}, status = {s}")

Found 6 local vertices for θ = 0.490874

--- Vertex #1: (r0 = 0.526058, r1 = 0.000000) ---
  Level 0           : primal = 1.3014515282, dual = 1.3014515246, status = optimal
  Level AA          : primal = 1.2152084395, dual = 1.2152084330, status = optimal
  Level BB          : primal = 1.3014515306, dual = 1.3014515140, status = optimal
  Level AB          : primal = 1.0106080574, dual = 1.0106080534, status = optimal
  Level BB+AB       : primal = 1.0106080615, dual = 1.0106080517, status = optimal
  Level ABB         : primal = 1.2210094966, dual = 1.2210094873, status = optimal
  Level AAB         : primal = 1.1235866248, dual = 1.1235866227, status = optimal
  Level AB+ABB      : primal = 1.0000000017, dual = 0.9999999983, status = optimal
  Level AA+AB+ABB   : primal = 1.0000000008, dual = 0.9999999994, status = optimal
  Level BB+AB+ABB   : primal = 1.0000000068, dual = 0.9999999957, status = optimal
  Level AA+BB+AB+ABB: primal = 1.0000000020, dual = 0.9999999984, status = opti

In [16]:
theta = np.pi / 8 + np.pi / (64)
tolerance = 1e-6
levels = [0, 'AA', 'BB', 'AB', 'BB+AB', 'ABB', 'AAB', 'AB+ABB', 'AA+AB+ABB', 'BB+AB+ABB', 'AA+BB+AB+ABB', 'AB+AAB+ABB']

with open("vertexCoords.json", "r") as f:
    local_vertices_data = json.load(f)

local_verts = None
for theta_value, verts in local_vertices_data:
    if abs(theta_value - theta) < tolerance:
        local_verts = np.array(verts)
        break

if local_verts is None:
    raise ValueError(f"No local polytope found for theta = {theta}")

print(f"Found {len(local_verts)} local vertices for θ = {theta:.6f}")

for i, (r0, r1) in enumerate(local_verts):
    print(f"\n--- Vertex #{i+1}: (r0 = {r0:.6f}, r1 = {r1:.6f}) ---")
    for lvl in levels:
        p, d, s = test_point_quantum_bound(theta, r0, r1, level=lvl)
        print(f"  Level {lvl:<12}: primal = {p:.10f}, dual = {d:.10f}, status = {s}")

Found 6 local vertices for θ = 0.441786

--- Vertex #1: (r0 = 0.602949, r1 = 0.000000) ---
  Level 0           : primal = 1.3542367433, dual = 1.3542367396, status = optimal
  Level AA          : primal = 1.2339448488, dual = 1.2339448348, status = optimal
  Level BB          : primal = 1.3542367448, dual = 1.3542367358, status = optimal
  Level AB          : primal = 1.0069550667, dual = 1.0069550627, status = optimal
  Level BB+AB       : primal = 1.0069550696, dual = 1.0069550627, status = optimal
  Level ABB         : primal = 1.2914740543, dual = 1.2914740468, status = optimal
  Level AAB         : primal = 1.1714021705, dual = 1.1714021642, status = optimal
  Level AB+ABB      : primal = 1.0000000054, dual = 0.9999999968, status = optimal
  Level AA+AB+ABB   : primal = 1.0000000013, dual = 0.9999999991, status = optimal
  Level BB+AB+ABB   : primal = 1.0000000010, dual = 0.9999999993, status = optimal
  Level AA+BB+AB+ABB: primal = 1.0000000021, dual = 0.9999999985, status = opti

## Extracting the certificate at $T_{1+A+B+AB+ABB}$

We are able to reach $\beta_T$ at this level of the hierarchy.

In [5]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)
    
    # AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()   # extra monomials

Theta = np.pi/4
r0 = 1 - 1/np.sqrt(2) # p1 = r0
r1 = 0 # p4 = 1/2 r1
s = np.sqrt(2)
sin_2T = np.sin(2 * Theta)
sin_4T = np.sin(4 * Theta)
cos_2T = np.cos(2 * Theta)
cos_4T = np.cos(4 * Theta)
cos_6T = np.cos(6 * Theta)

obj = (-1/(2*(3 - cos_4T))*0.5*r1*(-4*B[1][0]*(-3 + cos_4T) + s*(-5*cos_2T + cos_6T)*A[0][0]*B[1][0] 
    + s*(-5*cos_2T + cos_6T)*A[1][0]*B[1][0] + 2*s*A[0][0]*(-3 + cos_4T)*sin_2T - 2*s*A[1][0]*(-3 + cos_4T)*sin_2T) 
    - 1/(2*(3 - cos_4T))*(-4*B[0][0]*cos_2T - s*A[0][0]*B[0][0] + s*cos_4T*A[0][0]*B[0][0] - s*A[1][0]*B[0][0] 
    + s*cos_4T*A[1][0]*B[0][0] - 2*s*A[0][0]*B[1][0]*sin_2T + 2*s*A[1][0]*B[1][0]*sin_2T) 
    - 1/(2*(3 - cos_4T))*r0*(s*A[0][0]*(-3 + cos_4T) + s*A[1][0]*(-3 + cos_4T) - 4*B[0][0]*(-1 + cos_4T) 
    + 2*s*cos_2T*A[0][0]*B[0][0] + 2*s*cos_2T*A[1][0]*B[0][0] - 2*s*cos_2T*A[0][0]*B[1][0]*sin_2T + s*A[1][0]*B[1][0]*sin_4T))


sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)

1.0000000083684621 0.9999999939884063 optimal


In [6]:
b = sdp.y_mat[0]
print(b)
np.savetxt("Gamma.csv", b, delimiter=",")

[[ 2.01690141e-01 -4.70151340e-02 -4.70151298e-02  7.32242705e-02
   1.76690225e-11 -8.23302327e-02 -6.02862953e-02 -8.23302286e-02
   6.02862913e-02 -4.07424750e-04  4.35463592e-03  4.07424712e-04
  -4.35463183e-03]
 [-4.70151340e-02  8.97906600e-02  8.84037876e-03 -7.81067059e-02
  -5.43191544e-02  4.18571766e-02  1.80198513e-02  1.52269678e-02
   8.61489393e-03  1.30811715e-02  5.10049391e-03 -1.30811660e-02
  -9.23367549e-03]
 [-4.70151298e-02  8.84037876e-03  8.97906641e-02 -7.81067058e-02
   5.43191585e-02  1.52269649e-02 -8.61489395e-03  4.18571737e-02
  -1.80198513e-02  1.30811660e-02  9.23367675e-03 -1.30811715e-02
  -5.10049690e-03]
 [ 7.32242705e-02 -7.81067059e-02 -7.81067058e-02  1.98380116e-01
   8.49680391e-11 -5.21813296e-02  4.05731654e-04 -5.21813297e-02
  -4.05731572e-04 -6.21712454e-02  4.00484575e-14  6.21712453e-02
   9.17409352e-13]
 [ 1.76690225e-11 -5.43191544e-02  5.43191585e-02  8.49680391e-11
   9.99295307e-02 -4.35294283e-03 -4.35692686e-03  4.35293869e-03


## Victor's Bell expression, $T_{1+A+B+AB+ABB'}$

In [18]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    #AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()    # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
r0 = 1 - 1/np.sqrt(2)
r1 = 0

# Victor's Bell expression
obj = r0*((A[0][0] + A[1][0])/np.sqrt(2) - B[0][0]) + r1*((A[0][0] - A[1][0])/np.sqrt(2) - B[1][0]) + 1/(2*np.sqrt(2))*(A[0][0]*B[0][0] + A[1][0]*B[0][0] + A[0][0]*B[1][0] - A[1][0]*B[1][0])

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)
sosvictor = sdp.get_sos_decomposition()

1.00000000835675 0.9999999939876847 optimal


We want to extract the dual solution in the monomial basis:

In [19]:
Gamma = sdp.x_mat[0]
Wmon = sdp.y_mat[0]
print(Wmon)

np.savetxt("Wmon.csv", Wmon, delimiter=",")
np.savetxt("Gamma.csv", Gamma, delimiter=",")

[[ 2.01690088e-01 -4.70151546e-02 -4.70150952e-02  7.32242701e-02
   2.33116392e-10 -8.23302791e-02 -6.02862652e-02 -8.23302206e-02
   6.02862070e-02 -4.07413137e-04  4.35468180e-03  4.07412623e-04
  -4.35462303e-03]
 [-4.70151546e-02  8.97906867e-02  8.84035154e-03 -7.81066671e-02
  -5.43192393e-02  4.18571525e-02  1.80198594e-02  1.52269742e-02
   8.61485540e-03  1.30811542e-02  5.10053686e-03 -1.30811489e-02
  -9.23365298e-03]
 [-4.70150952e-02  8.84035154e-03  8.97907460e-02 -7.81066650e-02
   5.43192984e-02  1.52269320e-02 -8.61485567e-03  4.18571107e-02
  -1.80198596e-02  1.30811481e-02  9.23367103e-03 -1.30811538e-02
  -5.10057960e-03]
 [ 7.32242701e-02 -7.81066671e-02 -7.81066650e-02  1.98379981e-01
   1.07251209e-09 -5.21813173e-02  4.05720622e-04 -5.21813178e-02
  -4.05719524e-04 -6.21711906e-02  1.14861721e-12  6.21711897e-02
   7.48866333e-13]
 [ 2.33116392e-10 -5.43192393e-02  5.43192984e-02  1.07251209e-09
   9.99297193e-02 -4.35298928e-03 -4.35691852e-03  4.35292993e-03


## Victor's Bell expression reverted, $T_{1+A+B+AB+ABB}$

In [7]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    #AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    
    return monos[:]

moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()    # extra monomials

# Compute quantum value of Bell expression at (p1, p4) for a given Theta
r0 = 1 - 1/np.sqrt(2)
r1 = 0

# Victor's Bell expression
obj = r0*((B[0][0] + B[1][0])/np.sqrt(2) - A[0][0]) + r1*((B[0][0] - B[1][0])/np.sqrt(2) - A[1][0]) + 1/(2*np.sqrt(2))*(A[0][0]*B[0][0] + A[1][0]*B[0][0] + A[0][0]*B[1][0] - A[1][0]*B[1][0])

sdp = ncp.SdpRelaxation(ops, verbose=0, normalized=True, parallel=0)
sdp.get_relaxation(
    level=0,
    equalities = op_eqs[:],
    inequalities = op_ineqs[:],
    momentequalities = moment_eqs[:],
    momentinequalities = moment_ineqs[:],
    objective=-obj,
    substitutions = subs,
    extramonomials = extra_monos
)
sdp.solve(solver="mosek")
print(-sdp.primal, -sdp.dual, sdp.status)
sosvictor = sdp.get_sos_decomposition()

1.000891103574409 1.0008910972216816 optimal
